In [72]:
import requests
import pandas as pd
import numpy as np

In [142]:
resp_heroes_stats = requests.get(f'https://api.opendota.com/api/heroStats').json()

<h2>Making stats for one hero (anti-mage)

Required columns (stats for players which rank is at least Legend
- hero_id
- name
- roles
- 5_pick
- 5_win
- 6_pick
- 6_win
- 7_pick
- 7_win
- 8_pick
- 8_win

In [68]:
columns = ['hero_id', 'name', 'roles', '5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win', '8_pick', '8_win']

In [65]:
resp_heroes_stats[0]['roles'] = '-'.join(resp_heroes_stats[0]['roles'])
resp_heroes_stats[0]['roles']

'Carry-Escape-Nuker'

In [69]:
dataframe = pd.DataFrame([resp_heroes_stats[0]], columns=columns).set_index('hero_id')
dataframe.head()

,name,roles,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win
hero_id,,,,,,,,,,
1,npc_dota_hero_antimage,Carry-Escape-Nuker,24369,12382,10390,5364,4335,2198,1180,597


In [79]:
winrate_columns = columns[3:]
np.partition(winrate_columns, (2, 2))

array(['5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win', '8_pick',
       '8_win'], dtype='<U6')

In [80]:
for column in winrate_columns:
    if column.endswith('pick'):
        column_pick = dataframe[column]
        column_win = dataframe[column.split('_')[0]+'_win']
        winrate = column_win / column_pick
        dataframe[column.split('_')[0]+'_winrate'] = winrate

In [81]:
dataframe

,name,roles,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win,5_winrate,6_winrate,7_winrate,8_winrate
hero_id,,,,,,,,,,,,,,
1,npc_dota_hero_antimage,Carry-Escape-Nuker,24369,12382,10390,5364,4335,2198,1180,597,0.508105,0.516266,0.507036,0.505932


<h2>Make winrate stats for all Dota 2 heroes

In [143]:
columns = ['hero_id', 'name', 'roles', '5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win', '8_pick', '8_win']
winrate_columns = columns[3:]

common_dataframe = pd.DataFrame()
for hero in resp_heroes_stats:
    hero_ = hero.copy()
    hero_['roles'] = '-'.join(hero_['roles'])
    dataframe = pd.DataFrame([hero_], columns=columns).set_index('hero_id')
    for column in winrate_columns:
        if column.endswith('pick'):
            column_pick = dataframe[column]
            column_win = dataframe[column.split('_')[0]+'_win']
            winrate = column_win / column_pick
            dataframe[column.split('_')[0]+'_winrate'] = winrate
    common_dataframe = common_dataframe.append(dataframe)

In [146]:
# mean values for all rows
common_dataframe['mean_winrate'] = common_dataframe[['5_winrate', '6_winrate', '7_winrate', '8_winrate']].mean(axis=1)

In [147]:
common_dataframe.head()

,name,roles,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win,5_winrate,6_winrate,7_winrate,8_winrate,mean_winrate
hero_id,,,,,,,,,,,,,,,
1,npc_dota_hero_antimage,Carry-Escape-Nuker,24524,12479,10418,5317,4297,2152,1189,615,0.508848,0.510367,0.500815,0.517241,0.509318
2,npc_dota_hero_axe,Initiator-Durable-Disabler-Jungler-Carry,18891,9372,7354,3612,2679,1286,541,237,0.496109,0.491161,0.480030,0.438078,0.476345
3,npc_dota_hero_bane,Support-Disabler-Nuker-Durable,19306,10150,10446,5665,5827,3129,2979,1605,0.525743,0.542313,0.536983,0.538771,0.535953
4,npc_dota_hero_bloodseeker,Carry-Disabler-Jungler-Nuker-Initiator,9434,4741,4289,2169,1882,907,780,386,0.502544,0.505712,0.481934,0.494872,0.496266
5,npc_dota_hero_crystal_maiden,Support-Disabler-Nuker-Jungler,19206,9438,7919,3885,2835,1350,817,377,0.491409,0.490592,0.476190,0.461444,0.479909


In [148]:
common_dataframe.to_csv('heroes_winrate.csv')